Goal for 6/25
    - add logic for checking year
    - add logic for comparing education
    - add logic to parse work summary

In [49]:
from selenium import webdriver
import csv
import time
import random

In [60]:
search_list = []
#input = pd.read_csv('ten_sample.csv')
with open('ten_sample.csv', 'r') as input:
    for row in input:
        search_list.append(row)
search_list.pop(0)        

'"First\tLast\tClass"\n'

In [115]:
driver = webdriver.Chrome()

In [63]:
url = 'https://www.linkedin.com/'
username = 'hbb3xe@virginia.edu'
password = 'S+zMKr5o'

In [116]:
driver.get(url)

In [117]:
driver.find_element_by_id('login-email').send_keys(username)
driver.find_element_by_id('login-password').send_keys(password)

In [118]:
driver.find_element_by_name('submit').click()

In [119]:
driver.find_element_by_id('advanced-search').click()

In [105]:
school = 'University of Virginia'

In [120]:
scraped_info = []
for row in search_list:
    first, last, year = row.split('\t')
    first_name = first.replace('"', '')
    last_name = last.strip("\"\n")
    year = year.replace('"\n', '')
    print("Processing: {} {} {}".format(first, last, year))
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1.5, .5))
    
    driver.find_element_by_id('advs-firstName').clear()
    driver.find_element_by_id('advs-lastName').clear()
    driver.find_element_by_id('advs-school').clear()
    
    driver.find_element_by_id('advs-firstName').send_keys(first_name)
    driver.find_element_by_id('advs-lastName').send_keys(last_name)
    driver.find_element_by_id('advs-school').send_keys(school)
    driver.find_element_by_class_name('submit-advs').click()
    time.sleep(random.gauss(1.2, .5))
    
    num_results = driver.find_element_by_class_name('search-info').text.split('\n')[0]
    num_results = num_results.strip(" results")
    #num_results = int(num_results)
    
    results_summary = driver.find_element_by_id('results-container').text.split('\n')
    scraped_name = results_summary[0].strip('1st').strip('Premium Badge')
    summary_position = results_summary[1]
    
    full_name = first_name + " " + last_name
    name_match = "True" if scraped_name == full_name else "False"
    
    driver.find_element_by_css_selector('a.title.main-headline').click()
    time.sleep(random.gauss(1.5, .5))
    
    education_text = driver.find_element_by_id('background-education').text.lstrip('Education\n')
    school_match = "True" if school in education_text else "False"
    year_match = "True" if year in education_text else "False"
    parsed_education_text = education_text.split('\n')
    
    work_experience = driver.find_element_by_id('background-experience-container').text.lstrip('Experience\n')
    parsed_work_experience = work_experience.split('\n')
    
    driver.find_element_by_id('advanced-search').click()
    time.sleep(random.gauss(1.5, .5))
    
    info = (first_name, last_name, year, num_results, scraped_name, summary_position, parsed_education_text,
                parsed_work_experience, name_match, school_match, year_match)
    scraped_info.append(info)

Processing: "Sutton White 2016
Processing: "Kevin Hsu 2014


WebDriverException: Message: unknown error: Element is not clickable at point (350, 213). Other element would receive the click: <div id="voltron-overlay"></div>
  (Session info: chrome=51.0.2704.103)
  (Driver info: chromedriver=2.21.371459 (36d3d07f660ff2bc1bf28a75d1cdabed0983e7c4),platform=Mac OS X 10.11.5 x86_64)


In [114]:
driver.close()

In [88]:
with open('results.csv', 'w') as output:
    scraper_writer = csv.writer(output)
    scraper_writer.writerow(['Input First','Input Last','Input Year','No. Search Results','Search Results Name',
                             'Search Results Work Summary', 'Education Experience','Work Experience',
                            'Name Match','School Match','Year Match'])
    for row in scraped_info:
        scraper_writer.writerow(row)

[]